In [1]:
import pandas as pd
import plotly.express as px
from internproject.constants import ROOT_DATA_DIR

add_lp = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_add_lp.parquet")  # more details on the add liquidity events
remove_lp = pd.read_parquet(
    ROOT_DATA_DIR / "stETH_ETH_remove_lp.parquet"
)  # more details on the remove liquidity events
lp_transfers = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_lp_transfers.parquet")

In [2]:
null_address = "0x0000000000000000000000000000000000000000"

In [3]:
lp_transfers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-05 00:26:18,0x0000000000000000000000000000000000000000,0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a,0.000000,Transfer,11591275,88,148,0x4fdd5de055a85451535ae48f21353f922554d9416903...,2021-01-05 00:26:18
2021-01-05 11:26:21,0x0000000000000000000000000000000000000000,0x6ab615CF8deCFc488186E54066Fc10589C9293A3,0.160000,Transfer,11594223,141,250,0x16d97e10e582735b4731aaec2823f781edb3355fa9b2...,2021-01-05 11:26:21
2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18
2021-01-05 13:36:06,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x182B723a58739a9c974cFDB385ceaDb237453c28,69.999492,Transfer,11594833,50,43,0xacd73e0a596914d36650b57a0bf264cc8a8b63e053a8...,2021-01-05 13:36:06
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47
...,...,...,...,...,...,...,...,...,...
2023-06-21 00:00:11,0x99ac10631F69C753DDb595D074422a0922D9056B,0x182B723a58739a9c974cFDB385ceaDb237453c28,0.003230,Transfer,17524329,349,316,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11
2023-06-21 00:00:11,0x182B723a58739a9c974cFDB385ceaDb237453c28,0x989AEb4d175e16225E39E87d0D97A3360524AD80,0.003230,Transfer,17524329,349,318,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11
2023-06-21 00:00:11,0x989AEb4d175e16225E39E87d0D97A3360524AD80,0xF403C135812408BFbE8713b5A23a04b3D48AAE31,0.003230,Transfer,17524329,349,321,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11


### Understanding lp transfers

1. The "value" column in `lp_transfers` refers to the quantity of lp tokens
2. If the null address is in the "from" column, then it is adding liquidity to the pool. 
3. If the null address is in the "to" column, then it is removeing liquidity from the pool.
4. It is safe to ignore the other transfers

In [4]:
add_liquidity_transfers = lp_transfers[lp_transfers["from"] == null_address]
add_liquidity_transfers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-05 00:26:18,0x0000000000000000000000000000000000000000,0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a,0.000000,Transfer,11591275,88,148,0x4fdd5de055a85451535ae48f21353f922554d9416903...,2021-01-05 00:26:18
2021-01-05 11:26:21,0x0000000000000000000000000000000000000000,0x6ab615CF8deCFc488186E54066Fc10589C9293A3,0.160000,Transfer,11594223,141,250,0x16d97e10e582735b4731aaec2823f781edb3355fa9b2...,2021-01-05 11:26:21
2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47
2021-01-05 14:32:47,0x0000000000000000000000000000000000000000,0xb084F0AdB86BF6B001E540Ac05439D98A499ce1D,0.999754,Transfer,11595091,101,132,0xa3ffd6f2c890037b38c11d64e34e991fa1f31318e873...,2021-01-05 14:32:47
...,...,...,...,...,...,...,...,...,...
2023-06-20 13:17:47,0x0000000000000000000000000000000000000000,0x60F727BdeAD2ce49B00f2A2133Fc707b931D130B,0.000930,Transfer,17521155,157,419,0xf82ec22d0978797d3bb8428986d19bded9cac2cec209...,2023-06-20 13:17:47
2023-06-20 14:07:59,0x0000000000000000000000000000000000000000,0x93621DCA56fE26Cdee86e4F6B18E116e9758Ff11,0.000930,Transfer,17521403,16,113,0x1ca5708625a94e5a4365062ded1116c9d08b501f2d96...,2023-06-20 14:07:59
2023-06-20 14:52:23,0x0000000000000000000000000000000000000000,0x64bB6967b85F7084C367774bc840b4a4c4Afcd8E,0.005861,Transfer,17521623,50,55,0xeb436efb3b0de1c2adb18fa01f58b5d99ffebf63ff8c...,2023-06-20 14:52:23


In [5]:
withdraw_liquidity_transfers = lp_transfers[lp_transfers["to"] == null_address]
withdraw_liquidity_transfers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-05 17:12:33,0x3a63aA8AE11Ecaa82fC352d0390634E8030B2ffd,0x0000000000000000000000000000000000000000,2.000945,Transfer,11595786,133,295,0xbd58a23237562e0332aaa7a7668f8dd71e4aecddc95d...,2021-01-05 17:12:33
2021-01-05 17:46:38,0xaF297deC752c909092A117A932A8cA4AaaFF9795,0x0000000000000000000000000000000000000000,499.501433,Transfer,11595955,11,5,0xad790ed2cb537281ae0c7eaf1558b1494b869e7f7e1c...,2021-01-05 17:46:38
2021-01-05 21:03:54,0xb084F0AdB86BF6B001E540Ac05439D98A499ce1D,0x0000000000000000000000000000000000000000,0.999754,Transfer,11596894,75,96,0xc485ceb5703ddaf5393049b8fb71a06b20ee0c0b4ced...,2021-01-05 21:03:54
2021-01-06 03:36:23,0x89B8B20AE90328692cD367f75aaFadF55fd33E8B,0x0000000000000000000000000000000000000000,10.037649,Transfer,11598604,77,130,0xbebdcd6fa25430bea017919704fd3dcd12f55ef0f99b...,2021-01-06 03:36:23
2021-01-06 12:17:18,0xdE3E412d4fe3c9d90ac74d0A9B064951B39EEae4,0x0000000000000000000000000000000000000000,1.002055,Transfer,11600957,56,66,0x5a33f4f9e2e2a55f09141909f75b1b308d092183681c...,2021-01-06 12:17:18
...,...,...,...,...,...,...,...,...,...
2023-06-20 22:25:35,0xEa508F82728927454bd3ce853171b0e2705880D4,0x0000000000000000000000000000000000000000,0.000000,Transfer,17523858,45,95,0x6188d0177bb9a8a4a3bb5feb560477d6ec4642c9fde0...,2023-06-20 22:25:35
2023-06-20 22:33:11,0x46D5F548E02fdee3ea7E4168B7cF896429fE6fe7,0x0000000000000000000000000000000000000000,65.033669,Transfer,17523896,78,240,0x113260903e90d948871a8a0af2f03156c17781a455ea...,2023-06-20 22:33:11
2023-06-20 22:40:35,0x5aD15EBa9D4E9351414084Eb1dd1EACc2068A4A7,0x0000000000000000000000000000000000000000,51.167100,Transfer,17523933,42,101,0x39af71a3003ae573c6739b9679bede548ea7cc53a6b0...,2023-06-20 22:40:35


In [6]:
withdraw_liquidity_transfers['value'] = withdraw_liquidity_transfers['value']* -1
withdraw_liquidity_transfers['value']

/var/folders/d9/vxfbfh9572v4lvm_hy19jbsh0000gn/T/ipykernel_53541/789938084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  withdraw_liquidity_transfers['value'] = withdraw_liquidity_transfers['value']* -1


timestamp
2021-01-05 17:12:33     -2.000945
2021-01-05 17:46:38   -499.501433
2021-01-05 21:03:54     -0.999754
2021-01-06 03:36:23    -10.037649
2021-01-06 12:17:18     -1.002055
                          ...    
2023-06-20 22:25:35     -0.000000
2023-06-20 22:33:11    -65.033669
2023-06-20 22:40:35    -51.167100
2023-06-20 23:28:11     -0.000000
2023-06-21 00:00:11     -0.003230
Name: value, Length: 27851, dtype: float64

In [7]:
add_liquidity_transfers['user'] = add_liquidity_transfers['to']
withdraw_liquidity_transfers['user'] = withdraw_liquidity_transfers['from']

transfer_df = pd.concat([add_liquidity_transfers, withdraw_liquidity_transfers])
transfer_df.columns

/var/folders/d9/vxfbfh9572v4lvm_hy19jbsh0000gn/T/ipykernel_53541/3897710871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_liquidity_transfers['user'] = add_liquidity_transfers['to']
/var/folders/d9/vxfbfh9572v4lvm_hy19jbsh0000gn/T/ipykernel_53541/3897710871.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  withdraw_liquidity_transfers['user'] = withdraw_liquidity_transfers['from']


Index(['from', 'to', 'value', 'event', 'block', 'transaction_index',
       'log_index', 'hash', 'timestamp', 'user'],
      dtype='object')

In [8]:
transfer_df['user'].value_counts().head(100).tail(1)

0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D    25
Name: user, dtype: int64

In [9]:
transfer_df['user'] == '0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D'

timestamp
2021-01-05 00:26:18    False
2021-01-05 11:26:21    False
2021-01-05 13:35:18    False
2021-01-05 13:48:47    False
2021-01-05 14:32:47    False
                       ...  
2023-06-20 22:25:35    False
2023-06-20 22:33:11    False
2023-06-20 22:40:35    False
2023-06-20 23:28:11    False
2023-06-21 00:00:11    False
Name: user, Length: 62068, dtype: bool

In [10]:
transfer_df_1person = transfer_df[transfer_df["user"] == '0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D']
transfer_df_1person

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,
2022-09-20 07:55:35,0x0000000000000000000000000000000000000000,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D,1.919230,Transfer,15573341,40,72,0x1e5c9a123acda0980746b46f5da597988a26550f1214...,2022-09-20 07:55:35,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D
2022-09-25 09:18:59,0x0000000000000000000000000000000000000000,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D,58.129555,Transfer,15609444,41,39,0x94af6df6bf449716cdbdb589ef9b53f58eb3ab48e7ce...,2022-09-25 09:18:59,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D
2022-09-25 09:25:47,0x0000000000000000000000000000000000000000,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D,19.058853,Transfer,15609478,116,138,0x852a417275bad6b0bf0de4fe4b93aeab78b00f6f7fae...,2022-09-25 09:25:47,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D
2022-09-26 04:57:35,0x0000000000000000000000000000000000000000,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D,47.623412,Transfer,15615307,101,133,0xfbdab7cf41f588272f93626a70948bfcbf627791d057...,2022-09-26 04:57:35,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D
2022-09-29 00:28:35,0x0000000000000000000000000000000000000000,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D,47.571981,Transfer,15635420,137,89,0x08f6ad11cf3dba3ce494ac7a7ec6556bb4c2b7844e27...,2022-09-29 00:28:35,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D
2022-10-03 05:15:11,0x0000000000000000000000000000000000000000,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D,76.050830,Transfer,15665474,17,31,0xd62e59bfb3efa44f822ec323ef94373abf505a8b61a9...,2022-10-03 05:15:11,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D
2022-10-08 00:34:11,0x0000000000000000000000000000000000000000,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D,4.478024,Transfer,15699817,73,138,0x14acf2dcd21f907c8fcabbf2601e13aa382050b0b629...,2022-10-08 00:34:11,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D
2022-10-17 08:58:59,0x0000000000000000000000000000000000000000,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D,56.099594,Transfer,15766791,232,482,0x484427a2ce8b70141a90da00f8abd9041b5e0ce6518a...,2022-10-17 08:58:59,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D
2022-10-18 06:49:23,0x0000000000000000000000000000000000000000,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D,21.862393,Transfer,15773306,89,215,0x1952d00106171a5f7ea6d031e9ddef86fe05c135e5b4...,2022-10-18 06:49:23,0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D


In [11]:
# finding balance over time 
transfer_df_1person['value'].cumsum().round(3).values
transfer_df_1person['balance_over_time'] = transfer_df_1person['value'].cumsum().round(3).values
transfer_df_1person['balance_over_time']

/var/folders/d9/vxfbfh9572v4lvm_hy19jbsh0000gn/T/ipykernel_53541/400335441.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transfer_df_1person['balance_over_time'] = transfer_df_1person['value'].cumsum().round(3).values


timestamp
2022-09-20 07:55:35      1.919
2022-09-25 09:18:59     60.049
2022-09-25 09:25:47     79.108
2022-09-26 04:57:35    126.731
2022-09-29 00:28:35    174.303
2022-10-03 05:15:11    250.354
2022-10-08 00:34:11    254.832
2022-10-17 08:58:59    310.931
2022-10-18 06:49:23    332.794
2022-10-19 03:17:11    365.091
2022-10-30 08:11:59    374.383
2022-10-31 07:41:47    380.261
2022-11-20 20:47:47    436.036
2022-12-05 05:40:59    625.921
2023-03-18 03:48:59    780.561
2023-04-24 11:46:59    798.313
2022-10-26 01:27:11    786.542
2022-10-30 08:09:47    766.756
2022-11-20 20:39:11    710.963
2022-11-20 20:49:35    655.187
2022-12-01 04:37:23    362.276
2023-02-26 09:05:59    342.276
2023-03-16 06:53:23    172.392
2023-03-21 06:11:47     17.752
2023-05-03 08:24:35     -0.000
Name: balance_over_time, dtype: float64

In [12]:
# amount of liquidity over time 
px.bar(transfer_df_1person['value'].cumsum().round(3).values)

In [13]:
# repeating process above for all unique users in transfer df
transfer_df.sort_index(inplace=True)
transfer_df['cumsol_col'] = transfer_df.groupby(['user'])['value'].cumsum()
pivot_df = transfer_df.pivot_table(values='cumsol_col', index=transfer_df.index, columns='user')


In [30]:
pivot_df = pivot_df.ffill()

In [33]:
px.line(pivot_df['0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D'])

### when did this person enter and leave

In [14]:
example_person = "0x80ee7c0e1e59929823eD9B22e34538b226967109"
add_liquidity_transfers[add_liquidity_transfers["to"] == example_person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,
2021-01-12 22:16:32,0x0000000000000000000000000000000000000000,0x80ee7c0e1e59929823eD9B22e34538b226967109,1.999396,Transfer,11642841,30,51,0xb03636820959b9641a5c4c39676967180d7629a9bc30...,2021-01-12 22:16:32,0x80ee7c0e1e59929823eD9B22e34538b226967109
2021-01-12 22:31:11,0x0000000000000000000000000000000000000000,0x80ee7c0e1e59929823eD9B22e34538b226967109,0.999698,Transfer,11642902,120,187,0xcf8723e0451638d3f68a12395facc8c0f1ed2178a90f...,2021-01-12 22:31:11,0x80ee7c0e1e59929823eD9B22e34538b226967109


In [15]:
withdraw_liquidity_transfers[withdraw_liquidity_transfers["from"] == example_person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,
2021-01-14 13:13:07,0x80ee7c0e1e59929823eD9B22e34538b226967109,0x0000000000000000000000000000000000000000,-2.999095,Transfer,11653414,210,243,0x642a0a0273035551b6fa585c8615e129bc05772c9277...,2021-01-14 13:13:07,0x80ee7c0e1e59929823eD9B22e34538b226967109


This person entered twice at 2021-01-12 and then left at 2021-01-14. So they were only in for 3 days.

### Open Problems

Here are some problems you could look at. Remember that the focus is figuring out how changing incentive APR affacts LPs entering and leaving the pool

1. How does the amount of ETH value added influence the duration of a person's stay in the pool? (does this relationship change over time?)
2. Among those who fully exited the pool, what was the difference between the starting and ending Annual Percentage Rate (APR)?
3. What was the average duration of people's participation in the pool?
4. How frequently did people perform actions such as adding liquidity or withdrawing liquidity? Did they break up these actions into smaller increments?
5. Can the participants be clustered into meaningful groups based on their LP sizes and durations?
6. Are there any notable thresholds or breakpoints at which people exhibit different behaviors? eg. after a person has been in a pool for > 1 month they almost never leave
7. It is important to pay special attention to participants with large LPs. How do the people who have added and withdrawn large amounts of liquidity differ?

# Elasticity Subgroups

##### LP's who provided over 100 ETH 

In [16]:
# sum up tokens provided for each coin and select lp's that provided over 100 
add_lp['total_tokens'] = add_lp['token_amounts_0'] + add_lp['token_amounts_1']  
add_over_100_subgroup = add_lp.loc[add_lp['total_tokens'] > 100] 

In [34]:
# set fees as price 
add_over_100_subgroup['fees_sum'] = add_over_100_subgroup['fees_0'] + add_over_100_subgroup['fees_1']
add_over_100_subgroup['prev_fees_sum'] = add_over_100_subgroup['fees_sum'].shift(-30)

# set token supply as supply
add_over_100_subgroup['prev_token_supply'] = add_over_100_subgroup['token_supply'].shift(-30)

# set variables for elasticity formula
q2 = add_over_100_subgroup['token_supply']
q1 = add_over_100_subgroup['prev_token_supply']
p2 = add_over_100_subgroup['fees_sum']
p1 = add_over_100_subgroup['prev_fees_sum']

# calculate elasticity 
percent_change_supply = (q2 - q1) / ((q2 + q1) / 2) * 100 
percent_change_price = (p2 - p1) / ((p2 + p1) / 2) * 100 
add_over_100_subgroup['elasticity'] = percent_change_supply / percent_change_price 
add_over_100_subgroup['elasticity']

# disable warning messages 
import warnings 
warnings.filterwarnings('ignore')

In [18]:
add_over_100_subgroup[['token_supply', 'prev_token_supply', 'fees_sum', 'prev_fees_sum']]

,token_supply,prev_token_supply,fees_sum,prev_fees_sum
timestamp,,,,
2021-01-05 13:48:47,2070.149490,18325.359354,1.080910e-05,0.003046
2021-01-05 14:41:13,6793.327970,18629.341514,4.469827e-05,0.000025
2021-01-05 14:57:05,10810.190861,18772.823439,3.835418e-03,0.012856
2021-01-05 16:59:23,10995.288561,19911.025304,1.491526e-02,0.000251
2021-01-05 17:29:36,11492.789049,25195.675884,5.092017e-02,0.002499
...,...,...,...,...
2023-06-13 02:57:59,530519.880924,NaN,2.389803e-02,NaN
2023-06-13 16:25:35,526656.497122,NaN,6.793229e-08,NaN
2023-06-16 15:02:47,496134.044774,NaN,1.014533e-02,NaN


In [19]:
px.line(add_over_100_subgroup['elasticity'])

##### LP's that added liquidity when APR is greater than 5.08%

In [20]:
stETH_ETH_apr_df = pd.read_csv(
    ROOT_DATA_DIR
    / 'daily_snapshots/0xDC24316b9AE028F1497c275EB9192a3Ea0f67022_apr_df.csv',
    index_col=0,
    parse_dates=True,
).dropna()

In [21]:
# finding rolled avg apr from og steth eth dataframe and set it as price 
stETH_ETH_apr_df['cur_naive_apr'] = stETH_ETH_apr_df[['raw_base_apr', 'raw_fee_apr', 'raw_incentive_apr', 'raw_price_apr']].sum(axis=1)
stETH_ETH_apr_df['rolling_cur_naive_apr'] = stETH_ETH_apr_df['cur_naive_apr'].rolling(7).mean()
stETH_ETH_apr_df['rolling_cur_naive_apr'].median()

5.086284024877622

In [22]:
import plotly.express as px
added_when_apr_over_med = stETH_ETH_apr_df.loc[stETH_ETH_apr_df['rolling_cur_naive_apr'] > stETH_ETH_apr_df['rolling_cur_naive_apr'].median()]
px.histogram(added_when_apr_over_med['block'].diff())


In [23]:
px.line(added_when_apr_over_med['rolling_cur_naive_apr'])

In [24]:
added_when_apr_over_med['date_only'] = added_when_apr_over_med.index
added_when_apr_over_med.index

DatetimeIndex(['2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19',
               '2022-10-20', '2022-10-21', '2022-10-22', '2022-10-23',
               '2022-10-24', '2022-10-25',
               ...
               '2023-02-26', '2023-02-27', '2023-02-28', '2023-03-01',
               '2023-03-14', '2023-03-15', '2023-03-16', '2023-03-17',
               '2023-03-18', '2023-03-19'],
              dtype='datetime64[ns]', name='timestamp', length=124, freq=None)

In [25]:
add_liquidity_transfers['date_only'] = pd.to_datetime(add_liquidity_transfers.index.date)

# Find the subset of rows where the date columns match between added_when_apr_over_med and add_liquidity_transfers
same_date_df = add_liquidity_transfers[add_liquidity_transfers['date_only'].isin(added_when_apr_over_med.index)]

# combine added and removed (bar graph)!not~(.no.nonot)
same_date_df



,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,date_only
timestamp,,,,,,,,,,,
2022-10-16 00:17:59,0x0000000000000000000000000000000000000000,0xF0CAE48484cb8e2b95FB4E3fC005012887068255,0.411512,Transfer,15757046,61,148,0xbc31d72abd8c7821f1db72788760baf4d73f8a2d2732...,2022-10-16 00:17:59,0xF0CAE48484cb8e2b95FB4E3fC005012887068255,2022-10-16
2022-10-16 02:03:11,0x0000000000000000000000000000000000000000,0x5Be876Ed0a9655133226BE302ca6f5503E3DA569,85.614299,Transfer,15757568,181,517,0x59bd9e3c70e5668e8c62199252143b574939d342896d...,2022-10-16 02:03:11,0x5Be876Ed0a9655133226BE302ca6f5503E3DA569,2022-10-16
2022-10-16 03:39:11,0x0000000000000000000000000000000000000000,0xa6F222A2eA6a1CFAFa48Ff0014EE4fE277580Aa0,10.206916,Transfer,15758046,53,149,0x315a49d9c52163bab137df2ace958aff8880013e4143...,2022-10-16 03:39:11,0xa6F222A2eA6a1CFAFa48Ff0014EE4fE277580Aa0,2022-10-16
2022-10-16 04:42:11,0x0000000000000000000000000000000000000000,0xa6F222A2eA6a1CFAFa48Ff0014EE4fE277580Aa0,0.382110,Transfer,15758359,198,525,0x799c111146a98a4702ceeac5dbeda8396f0ca5daf62d...,2022-10-16 04:42:11,0xa6F222A2eA6a1CFAFa48Ff0014EE4fE277580Aa0,2022-10-16
2022-10-16 04:48:47,0x0000000000000000000000000000000000000000,0x4334703B0B74E2045926f82F4158A103fCE1Df4f,2.613165,Transfer,15758392,140,279,0xdd6d8ec6c7528fc490a2c73eea6ddff9c1bc8da44ed3...,2022-10-16 04:48:47,0x4334703B0B74E2045926f82F4158A103fCE1Df4f,2022-10-16
...,...,...,...,...,...,...,...,...,...,...,...
2023-03-19 22:03:59,0x0000000000000000000000000000000000000000,0x638B129Ab6d241Ec2F890Aa3af197bF84b5FF1bB,9.434859,Transfer,16864605,6,21,0x7ee5425c100110c09209d4af4f522d8b46251cf5772c...,2023-03-19 22:03:59,0x638B129Ab6d241Ec2F890Aa3af197bF84b5FF1bB,2023-03-19
2023-03-19 22:49:11,0x0000000000000000000000000000000000000000,0x505F11278081C89F2d29b890168F260d299B42E2,0.046821,Transfer,16864829,144,365,0x36d9adbeed8ad1451c02bc072911f2951ba498f3377f...,2023-03-19 22:49:11,0x505F11278081C89F2d29b890168F260d299B42E2,2023-03-19
2023-03-19 23:03:47,0x0000000000000000000000000000000000000000,0xf27c4dB492dc4C1E6fA82858170b8d845e8Bb601,93.667583,Transfer,16864902,16,110,0x2ab93908a3ddea8f26b68e6c4da9c11b9ec270425bbd...,2023-03-19 23:03:47,0xf27c4dB492dc4C1E6fA82858170b8d845e8Bb601,2023-03-19


In [26]:
same_date_df

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,date_only
timestamp,,,,,,,,,,,
2022-10-16 00:17:59,0x0000000000000000000000000000000000000000,0xF0CAE48484cb8e2b95FB4E3fC005012887068255,0.411512,Transfer,15757046,61,148,0xbc31d72abd8c7821f1db72788760baf4d73f8a2d2732...,2022-10-16 00:17:59,0xF0CAE48484cb8e2b95FB4E3fC005012887068255,2022-10-16
2022-10-16 02:03:11,0x0000000000000000000000000000000000000000,0x5Be876Ed0a9655133226BE302ca6f5503E3DA569,85.614299,Transfer,15757568,181,517,0x59bd9e3c70e5668e8c62199252143b574939d342896d...,2022-10-16 02:03:11,0x5Be876Ed0a9655133226BE302ca6f5503E3DA569,2022-10-16
2022-10-16 03:39:11,0x0000000000000000000000000000000000000000,0xa6F222A2eA6a1CFAFa48Ff0014EE4fE277580Aa0,10.206916,Transfer,15758046,53,149,0x315a49d9c52163bab137df2ace958aff8880013e4143...,2022-10-16 03:39:11,0xa6F222A2eA6a1CFAFa48Ff0014EE4fE277580Aa0,2022-10-16
2022-10-16 04:42:11,0x0000000000000000000000000000000000000000,0xa6F222A2eA6a1CFAFa48Ff0014EE4fE277580Aa0,0.382110,Transfer,15758359,198,525,0x799c111146a98a4702ceeac5dbeda8396f0ca5daf62d...,2022-10-16 04:42:11,0xa6F222A2eA6a1CFAFa48Ff0014EE4fE277580Aa0,2022-10-16
2022-10-16 04:48:47,0x0000000000000000000000000000000000000000,0x4334703B0B74E2045926f82F4158A103fCE1Df4f,2.613165,Transfer,15758392,140,279,0xdd6d8ec6c7528fc490a2c73eea6ddff9c1bc8da44ed3...,2022-10-16 04:48:47,0x4334703B0B74E2045926f82F4158A103fCE1Df4f,2022-10-16
...,...,...,...,...,...,...,...,...,...,...,...
2023-03-19 22:03:59,0x0000000000000000000000000000000000000000,0x638B129Ab6d241Ec2F890Aa3af197bF84b5FF1bB,9.434859,Transfer,16864605,6,21,0x7ee5425c100110c09209d4af4f522d8b46251cf5772c...,2023-03-19 22:03:59,0x638B129Ab6d241Ec2F890Aa3af197bF84b5FF1bB,2023-03-19
2023-03-19 22:49:11,0x0000000000000000000000000000000000000000,0x505F11278081C89F2d29b890168F260d299B42E2,0.046821,Transfer,16864829,144,365,0x36d9adbeed8ad1451c02bc072911f2951ba498f3377f...,2023-03-19 22:49:11,0x505F11278081C89F2d29b890168F260d299B42E2,2023-03-19
2023-03-19 23:03:47,0x0000000000000000000000000000000000000000,0xf27c4dB492dc4C1E6fA82858170b8d845e8Bb601,93.667583,Transfer,16864902,16,110,0x2ab93908a3ddea8f26b68e6c4da9c11b9ec270425bbd...,2023-03-19 23:03:47,0xf27c4dB492dc4C1E6fA82858170b8d845e8Bb601,2023-03-19


In [27]:
# set price variables 
p2 = added_when_apr_over_med['rolling_cur_naive_apr']
p1 = added_when_apr_over_med['rolling_cur_naive_apr'].shift(-30)

# set supply variables 
q2 = add_liquidity_transfers['value']
q1 = add_liquidity_transfers['value'].shift(-30)

# calculate elasticity
percent_change_supply = (q2 - q1) / ((q2 + q1) / 2) * 100 
percent_change_price = (p2 - p1) / ((p2 + p1) / 2) * 100 
stETH_ETH_apr_df['elasticity'] = percent_change_supply / percent_change_price 


ValueError: cannot reindex on an axis with duplicate labels